Model built in Google Colab

In [ ]:
!pip install pytorch-lightning
!pip install sqlalchemy
!wget https://storage.googleapis.com/chesspic/datasets/2021-07-31-lichess-evaluations-37MM.db.gz
!gzip -d "2021-07-31-lichess-evaluations-37MM.db.gz"

In [ ]:
from sqlalchemy import *
import base64

db = create_engine("sqlite:///2021-07-31-lichess-evaluations-37MM.db")
LABEL_COUNT = 37164639

metadata = MetaData()
evaluations = Table(
    "evaluations",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("fen", Text),
    Column("binary", LargeBinary),
    Column("eval", Float),
)

metadata.create_all(db)


def binary_base64(binary):
    return base64.b64encode(binary)


In [ ]:
import numpy as np
from torch.utils.data import IterableDataset
from random import randrange


class EvaluationDataset(IterableDataset):
    def __init__(self, count):
        self.count = count

    def __iter__(self):
        return self

    def __next__(self):
        idx = randrange(self.count)
        return self[idx]

    def __len__(self):
        return self.count

    def __getitem__(self, idx):
        with db.connect() as conn:
            query = evaluations.select().where(evaluations.c.id == idx + 1)
            result = conn.execute(query)
            eval = result.fetchone()
            bin = np.frombuffer(eval.binary, dtype=np.uint8)
            bin = np.unpackbits(bin, axis=0).astype(np.single)
            eval.eval = max(eval.eval, -15)
            eval.eval = min(eval.eval, 15)
            ev = np.array([eval.eval]).astype(np.single)
        return {"binary": bin, "eval": ev}


dataset = EvaluationDataset(count=LABEL_COUNT)
